In [1]:
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from pathlib import Path

In [2]:
def create_folder_hierarchy(dest_path="./"):
    Path(dest_path + "/Flickr2K_Downsampled/HR").mkdir(parents=True, exist_ok=True)
    Path(dest_path + "/Flickr2K_Downsampled/LR").mkdir(parents=True, exist_ok=True)

In [3]:
create_folder_hierarchy()

In [4]:
def create_downsampled_images(source_path="./Flickr2K/Flickr2K_HR/", 
                              dest_path="./Flickr2K_Downsampled/", 
                              out_filename="flickr2k.csv"):
    file_list= [source_path + file for file in os.listdir(source_path) if file.endswith('.png')]
    
    col_names = ["LR_filename", "LR_filepath", "LR_dimensions",
                 "HR_filename", "HR_filepath", "HR_dimensions", 
                 "downsample_method"]
    data = {col:[] for col in col_names}
    N = len(file_list)
    for i, image_path in enumerate(tqdm(file_list)):
        image = Image.open(image_path)
        image_name = image_path.split("/")[-1]
        width, height = image.size
        
        # https://pillow.readthedocs.io/en/stable/handbook/concepts.html#concept-filters
        resize_method = Image.NEAREST
        resize_method_str = "NEAREST"
        if i in range((5*N//6), N):
            resize_method = Image.BOX
            resize_method_str = "BOX"
        if i in range((4*N//6), (5*N//6)):
            resize_method = Image.BILINEAR
            resize_method_str = "BILINEAR"
        if i in range((3*N//6), (4*N//6)):
            resize_method = Image.HAMMING
            resize_method_str = "HAMMING"
        if i in range((2*N//6), (3*N//6)):
            resize_method = Image.BICUBIC
            resize_method_str = "BICUBIC"
        if i in range((N//6), (2*N//6)):
            resize_method = Image.LANCZOS
            resize_method_str = "LANCZOS"
        
        hr_downsampled = image.resize((width//2, height//2), resample=resize_method)
        lr_downsampled = image.resize((width//4, height//4), resample=resize_method)
        
        hr_downsampled.save(dest_path + "HR/" + image_name)
        lr_downsampled.save(dest_path + "LR/train_" + image_name)
        
        hr_downsampled.close()
        lr_downsampled.close()
        
        data["LR_filename"].append("train_" + image_name)
        data["LR_filepath"].append(dest_path + "LR/train_" + image_name)
        data["LR_dimensions"].append(f"{width//4}x{height//4}")
        data["HR_filename"].append(image_name)
        data["HR_filepath"].append(dest_path+ "HR/" + image_name)
        data["HR_dimensions"].append(f"{width//2}x{height//2}")
        data["downsample_method"].append(resize_method_str)
        
    data = pd.DataFrame(data)
    data.to_csv(out_filename)

In [5]:
create_downsampled_images()

100%|██████████| 2650/2650 [33:50<00:00,  1.31it/s]
